In [ ]:
import numpy as np
np.__version__

'1.26.4'

### Load environment
Save steam_data.csv under "BUCKET_NAME/data" folder, change BUCKET_NAME in the following block

In [ ]:
from google.cloud import storage

# Set your GCP bucket name and file path
BUCKET_NAME = "eco395m-2024"
IN_BLOB_NAME = "data/steam_data.csv"
OUT_BLOB_NAME = "data/steam_clean_no_header.csv"

# Initialize a GCP storage client
client = storage.Client()

# Get the bucket
bucket = client.bucket(BUCKET_NAME)

# Read csv from the bucket
in_blob = bucket.blob(IN_BLOB_NAME)
# Download the csv content as a string
content = in_blob.download_as_text()

In [ ]:
import io
from tqdm import tqdm
import pandas as pd
from sentence_transformers import SentenceTransformer

tqdm.pandas()  # Progress bar

IN_PATH = io.StringIO(content)
csv_buffer = io.StringIO()  # Output

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


### Clean data

In [ ]:
# Read csv + Drop NA Title
# Clean price: Replace "Free" with "0" + Remove "$" and "," + Convert to float
# Clean date: coerce: If a date is not in "Jan 1, 2000" format, set it to NaT/NaN
# Combine Title, Description, Tags, and Features to one column "search_text"
df = (
    pd.read_csv(IN_PATH)
    .dropna(subset=["Title"])
    .assign(
        original_price=lambda df_: df_["Original Price"]
            .replace("Free", "0")
            .str.replace(r"[$,]", "", regex=True)
            .astype(float),
        release_date=lambda df_: pd.to_datetime(
            df_["Release Date"], format="%d %b, %Y", errors="coerce"
        ),
        search_text=lambda df_: df_["Title"]
            + " " + df_["Game Description"].fillna("")
            + " " + df_["Popular Tags"].fillna("")
            + " " + df_["Game Features"].fillna(""),
    )
)

### Sentence transformer

In [ ]:
# Initialize Sentence Transformer model
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
# Other more computationally complex models
# model = SentenceTransformer("distiluse-base-multilingual-cased-v1")
# model = SentenceTransformer("xlm-r-bert-base-nli-stsb-mean-tokens")
# model = SentenceTransformer("LaBSE")

# Add embedding column with progress bar
df["embedding"] = df["search_text"].progress_apply(
    lambda text: model.encode(text).tolist()
)

100%|██████████| 71699/71699 [1:33:48<00:00, 12.74it/s]


### Select columns + Write csv

In [ ]:
df[
    [
        "Title",
        "original_price",
        "release_date",
        "Game Description",
        "All Reviews Summary",
        "Developer",
        "Supported Languages",
        "Popular Tags",
        "Game Features",
        "embedding"
    ]
].to_csv(csv_buffer, header=False, index=False)

### Upload the modified csv back to the bucket

In [ ]:
out_blob = bucket.blob(OUT_BLOB_NAME)
out_blob.upload_from_string(csv_buffer.getvalue(), content_type="text/csv")